# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
#load the output_csv to 

weather_df = pd.read_csv("output_data/cities.csv")
weather_df

,city_id,city,country,date,lon,lat,temp,humd,cloud,wind
0,0,longyearbyen,SJ,1586974659,15.64,78.22,35.60,74,75,21.92
1,1,ribeira grande,PT,1586974753,-28.70,38.52,56.71,64,51,22.59
2,2,lalibela,ET,1586974753,39.03,12.03,61.09,40,69,8.84
3,3,saint-philippe,RE,1586974753,55.77,-21.36,77.00,69,6,21.92
4,4,alice town,BS,1586974753,-79.30,25.72,80.98,79,29,16.96
...,...,...,...,...,...,...,...,...,...,...
572,572,victor harbor,AU,1586974890,138.62,-35.57,66.20,77,75,11.41
573,573,havre-saint-pierre,CA,1586974890,-63.60,50.23,39.20,44,1,13.87
574,574,kalianget,ID,1586974890,113.93,-7.05,81.95,73,66,4.61
575,575,providencia,MX,1586974891,-111.58,28.72,87.01,15,5,8.19


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#keywords need to be passed in for layer of heatmap

In [15]:
# Configure gmaps
gmaps.configure(api_key=g_key)


In [16]:
# Store latitude, longitude and humidity in locations
locations = weather_df[["lat", "lon"]]
humidity = weather_df["humd"]

In [17]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=250,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#build out a large filter on your ideal conditions and drop all the rows that do not meet that criteria
#build out filter with .loc
#at the end of code use .dropna from a new data frame

In [18]:
# temperture preference is max = 90 and min = 70
# wind speed prefernce is less then 5 mph
# cloudiness preference is 0 

# & is an 'and' statement and | is a 'or' statement

preferences_df = weather_df.loc[(weather_df['temp'] >= 70)&(weather_df['temp'] < 90)&(weather_df['wind'] <= 5)&(weather_df['cloud'] == 0)].dropna()
preferences_df

,city_id,city,country,date,lon,lat,temp,humd,cloud,wind
188,188,guerrero negro,MX,1586974729,-114.06,27.98,77.16,26,0,4.41
389,389,santa fe,AR,1586974846,-60.70,-31.63,75.99,36,0,4.00
448,448,mamallapuram,IN,1586974861,80.17,12.63,86.00,79,0,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
#create an empty column with hotel
preferences_df['hotel name'] = ""
preferences_df

,city_id,city,country,date,lon,lat,temp,humd,cloud,wind,hotel name
188,188,guerrero negro,MX,1586974729,-114.06,27.98,77.16,26,0,4.41,
389,389,santa fe,AR,1586974846,-60.70,-31.63,75.99,36,0,4.00,
448,448,mamallapuram,IN,1586974861,80.17,12.63,86.00,79,0,3.36,


In [20]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [34]:

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in preferences_df.iterrows():
    
    lat = row["lat"]
    lng = row["lon"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    
    try:
#         print(json.dumps(name_address["results"][0]["name"], indent=4, sort_keys=True))
        preferences_df.loc[index, "hotel name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    
    
    #get the lat and lon from preferences df
    #add lat lon as a f string to our parameter dic, create key for each location, key is location and value is coordinates
    #define base url
    #make the request and print url, convert results to json
    #try and except statement (in try statement grab the first hotel and store name in df "hotel name" column)


In [35]:
preferences_df

,city_id,city,country,date,lon,lat,temp,humd,cloud,wind,hotel name
188,188,guerrero negro,MX,1586974729,-114.06,27.98,77.16,26,0,4.41,"Casa Laguna, Bed & Breakfast"
389,389,santa fe,AR,1586974846,-60.70,-31.63,75.99,36,0,4.00,Conquistador Hotel
448,448,mamallapuram,IN,1586974861,80.17,12.63,86.00,79,0,3.36,Hotel Ramakrishna


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in preferences_df.iterrows()]
locations = preferences_df[["lat", "lon"]]

In [42]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Casa Laguna, Bed & Breakfast</dd>\n<dt>City</dt><dd>guerrero negro</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Conquistador Hotel</dd>\n<dt>City</dt><dd>santa fe</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Ramakrishna</dd>\n<dt>City</dt><dd>mamallapuram</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n']

In [43]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig


# Display Map

Figure(layout=FigureLayout(height='420px'))